#### BART
- transformer 기반의 모델
- Encoder, Decoder 모두 사용하는 모델
    - Encoder: 입력되는 문장 데이터를 BERT 형식으로 이해 ( 순방향, 역방향 )
    - Decoder: 출력되는 요약문을 GPT 형식으로 생성
- 장문의 텍스트에서 요약 데이터를 생성하는 데 사용
- tokenizer로 `Sentencepiece` 이용
    - 인풋 tokenizer (문장 이해) 와 아웃풋 tokenizer (문장 생성) 를 따로 사용
        - 인풋 tokenizer에서는 document의 시작과 끝을 나타내는 특수 토큰들을 사용
        - 파이썬은 이식성, 확장성이 좋아, 아웃풋 tokenizer에서도 해당 특수 토큰들을 사용할 수도 있음.

In [5]:
# 문자형 데이터를 이용해 검증 지표를 만드는 데 사용되는 라이브러리
# !pip install evaluate rouge_score

In [2]:
import numpy as np
from datasets import Dataset, DatasetDict
import evaluate     # 문장 간의 검증 지표를 만들어주는 라이브러리
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 미리 학습된 모델 선정 (kobart - 아직 완벽하지는 않은 모델)
model_name = 'gogamza/kobart-base-v2'

In [7]:
# 학습에서 사용할 원문 데이터, 요약 데이터
train_docs = [
    '정부는 중소기업 세제 혜택과 R&D 세액 공제를 확대한다고 밝혔다',
    '해당 기업은 분기 실적에서 매출 성장을 기록했으며 신제품 출시를 예고했다'
]
train_sums = [
    '정부가 중소기업 지원을 확대한다',
    '기업이 실적 개선과 신제품 출시를 발표했다'
]

valid_docs = [
    '교육부가 디지털 교과서 도입을 추진한다고 발표했다'
]
valid_sums = [
    '교육부가 디지털 교과서 도입을 추진한다'
]

In [ ]:
# transformer 모델에서 list 형태의 데이터 사용 불가
# 따라서 Dataset 형태의 데이터를 불러와, Dataset들로 이루어진 Dict 형태로 만들어준다. (value가 Dataset)
raw_ds = DatasetDict(
    {
        # 학습 데이터
        'train': Dataset.from_dict(
            {
                'document': train_docs,     # 원본 문장 (input)
                'summary': train_sums       # 요약 문장 (output)
            }
        ),
        # 검증 데이터
        'validation': Dataset.from_dict(
            {
                'document': valid_docs,
                'summary': valid_sums
            }
        )
    }
)

# Dataset은 맨 앞 글자가 대문자이므로 class라는 것을 알 수 있고, class에는 속성(변수), 메서드(함수)가 존재한다.
# 그 중에는 pandas로부터 DataFrame 데이터를 가져와 Dataset으로 만들어주는 from_pandas라는 메서드가 있다.
# 그 외에도 Dataset.from_{파일형식} 은 해당 형식의 데이터를 Dataset으로 바꿔준다.

In [9]:
raw_ds

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 2
    })
    validation: Dataset({
        features: ['document', 'summary'],
        num_rows: 1
    })
})

In [11]:
# 토크나이저, 모델을 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast= True)
# use_fast(가속모드): BERT 모델은 RUST 언어 기반인데, 일반적으로 SKT의 KoBERT에선 use_fast가 만들어져있지 않아 이전 코드들에선 False로 뒀다.
# 그러나 KoBART 모델에는 RUST 기반의 가속모드가 존재하므로, 사용하는 편이 효율적.
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 입력/출력 문장의 최대 길이 설정
max_input_len = 512
max_target_len = 128

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


`sklearn`</br>
```python
X_train, X_test, y_train, y_test로 분할 + 성능을 올리기 위한 option
-> fit(X_train, y_train)
-> pred = predict(X_test)
-> f1_score(y_test, pred)
```
`transformer` (BERT 모델)
```python
tokenizer + 성능을 올리거나 부족한 token 길이를 채우기 위한(padding) option
-> Trainer - train, validation
-> predict


In [12]:
# 데이터 전처리, 토큰화
def tok_fn(batch):
    # 입력 데이터의 토큰화 -> 인코딩
    inputs = tokenizer(
        # batch의 인자값: 묶음형 데이터셋
        # raw_ds에서 인풋 데이터는 document의 데이터
        batch['document'],
        max_length = max_input_len,
        padding = 'max_length',     # 고정 길이 패딩 사용
        truncation = True           # 문장이 최대 길이(max_token의 개수)보다 긴 경우 자른다.
    )
    # 출력 데이터의 토큰화 -> 인코딩
    # 아웃풋 전용 토크나이저 사용
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch['summary'],
            max_length = max_target_len,
            padding = 'max_length',
            truncation = True
        )
    # input에서 사용한 토크나이저 != labels에서 사용한 토크나이저

    # padding 토큰을 -100으로 변경 (CrossEntropyloss에서 -100이라는 값을 무시)
    # tokenizer의 result:
    #   - input_ids: 단어 사전에 있는 인덱스의 값들로 인코딩된 데이터 (토큰화된 Input Text)
    #   - attention_mask: 각 토큰이 실제 단어인지(1) 패딩 토큰(0)인지 나타냄
    #   - token_type_ids: 문장의 위치

    # 값을 바꾸기 용이하도록 Dataset에서 numpy array 형태로 바꿈
    labels_ids = np.array(labels['input_ids'])  # 실제 문장에 대한 토큰값
    # 행렬에서 pad_token_id(사용하는 tokenizer에서 <PAD> 토큰의 위치)와 같은 값을 가진 데이터를 -100으로 변경
    labels_ids[ labels_ids == tokenizer.pad_token_id ] = -100
    # labels_ids를 inputs에 있는 labels에 대입
    inputs['labels'] = labels_ids.tolist()

    return inputs

In [13]:
# tok_fn 호출
# raw_ds의 데이터를 tok_fn에 대입
tokenized_ds = raw_ds.map(
    tok_fn,
    batched= True,
    remove_columns= ['document', 'summary']     # 필요 없는 컬럼 제거
)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1/1 [00:00<00:00, 224.75 examples/s]


In [14]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
})

In [15]:
# DataCollator: padding 토큰의 동적인 처리 + 모델 입력 형태 자동 구현
data_collator = DataCollatorForSeq2Seq(
    tokenizer= tokenizer,
    model= model
)

In [16]:
# 검증 지표 선택
rouge = evaluate.load('rouge')

In [17]:
def compute_metrics(eval_pred):
    preds, labels = eval_pred

    # 디코더의 역할: vocab의 인덱스로 구성된 리스트를 다시 문자 형태로 변환
    #   예: preds = [13, 17, 102, ...(사전 vocab=['A', 'B', ...]에서 해당 값의 위치)] 처럼 labels 데이터와 같은 형태로 만들어질 것.
    # padding 토큰을 무시하기 위해 -100으로 구성했던 것을, 원래의 id 값으로 전환
    labels = np.where(
        labels != -100, labels, tokenizer.pad_token_id
    )
    # 텍스트 디코딩
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)  # 특수 토큰들은 디코드하지 않음
    label_str = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 문장의 좌우에 공백이 존재하는 경우 다른 값으로 측정하기 때문에 좌우 공백 제거
    pred_str = [ doc.strip() for doc in pred_str ]
    label_str = [ doc.strip() for doc in label_str ]

    # Rouge 계산
    result = rouge.compute(
        predictions= pred_str,
        references= label_str,
        use_stemmer= True       # 점수 계산 시 단어의 어간을 기준으로 비교할 것인가?
    )

    # Rouge를 보기 편한 형태로 변경
    result = { k: round(v * 100, 2) for k, v in result.items() }

    return result

In [18]:
# Trainer 파라미터 값을 지정
args = Seq2SeqTrainingArguments(
    output_dir= "./kobart",
    eval_strategy= 'epoch',
    save_strategy= 'epoch',
    learning_rate= 5e-5,
    num_train_epochs= 10,           # 요약이 잘 나오게 하기 위해 많이 돌림 (5번 돌려보니 이상하게 나왔음)
    logging_steps= 10,

    # generate 설정 변경 (성능적인 부분)
    predict_with_generate= True,
    generation_max_length= 70,      # 출력 토큰의 최대 길이
    generation_num_beams= 4,        # 요약 데이터를 생성할 때 문장 후보 탐색 개수

    load_best_model_at_end= True,
    metric_for_best_model= 'rougeL',    # rouge1(단어별 확인), rouge2, rougeL(일반적. 가장 큰 경우)
    greater_is_better= True,
    report_to= []
)

In [20]:
# Trainer 생성
trainer = Seq2SeqTrainer(
    model= model,
    args= args,
    train_dataset= tokenized_ds['train'],
    eval_dataset= tokenized_ds['validation'],
    tokenizer= tokenizer,
    data_collator= data_collator,
    compute_metrics= compute_metrics
)
trainer.train()

C:\Users\student\AppData\Local\Temp\ipykernel_8896\1379372643.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,4.601643,0.000000,0.000000,0.000000,0.000000
2,No log,3.585189,0.000000,0.000000,0.000000,0.000000
3,No log,3.075991,0.000000,0.000000,0.000000,0.000000
4,No log,3.548694,0.000000,0.000000,0.000000,0.000000
5,No log,3.685046,0.000000,0.000000,0.000000,0.000000
6,No log,3.791951,0.000000,0.000000,0.000000,0.000000
7,No log,3.976944,0.000000,0.000000,0.000000,0.000000
8,No log,4.056381,0.000000,0.000000,0.000000,0.000000
9,No log,4.094729,0.000000,0.000000,0.000000,0.000000
10,2.527300,4.143718,0.000000,0.000000,0.000000,0.000000


c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'forced_eos_token_id': 1}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\student\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:666: Use

TrainOutput(global_step=10, training_loss=2.527338981628418, metrics={'train_runtime': 174.208, 'train_samples_per_second': 0.115, 'train_steps_per_second': 0.057, 'total_flos': 6097364582400.0, 'train_loss': 2.527338981628418, 'epoch': 10.0})

---
Test

In [22]:
test_text = "과학기술정보통신부는 초거대 AI 연구 인프라 지원을 강화한다고 밝혔다. 스타트업 새당으로 GPU 리소스를 확대 제공할 계획이다."

inputs = tokenizer(
    test_text,
    return_tensors = 'pt',
    truncation = True,
    max_length = max_input_len
)
inputs.pop('token_type_ids', None)

gen_ids = model.generate(
    **inputs.to(model.device),
    max_length = 20,
    num_beams = 4,
    do_sample = False
)

print(tokenizer.decode(gen_ids[0], skip_special_tokens=True))

하겠다고 밝혔다. 스타트업 새당으로 GPU 리소스를 확대 제공할 계획이다.
